In [23]:
from datasets import load_dataset

In [24]:
#dataset = load_dataset('json',data_files=['train.json','validation.json'])
dataset = load_dataset('json',data_files={'train':'../Datasets/train_gleif_bisnode_FULL.json','test':'../Datasets/test_gleif_bisnode_FULL.json'})

Using custom data configuration default-496197a86409d968
Reusing dataset json (C:\Users\liuyw\.cache\huggingface\datasets\json\default-496197a86409d968\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50)


  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
dataset_input = load_dataset('json',data_files={'input':'../Datasets/input_gleif_bisnode_FULL_final.json'})

Using custom data configuration default-2548fa9cfe6b59ea
Reusing dataset json (C:\Users\liuyw\.cache\huggingface\datasets\json\default-2548fa9cfe6b59ea\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'labels', 'sentence1', 'sentence2'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['idx', 'labels', 'sentence1', 'sentence2'],
        num_rows: 500
    })
})

In [25]:
dataset_input

DatasetDict({
    input: Dataset({
        features: ['idx', 'sentence1', 'sentence2'],
        num_rows: 38396
    })
})

In [26]:
from transformers import AutoTokenizer

In [28]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\liuyw/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at C:\Users

In [29]:
def tokenize_function(example):
    return tokenizer(example["sentence1"],example["sentence2"],padding="max_length",truncation=True,max_length=128)

In [30]:
tokenized_datasets = dataset.map(tokenize_function,batched=True)
print(tokenized_datasets.column_names)

Loading cached processed dataset at C:\Users\liuyw\.cache\huggingface\datasets\json\default-496197a86409d968\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50\cache-141189dc193e7dc4.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

{'train': ['attention_mask', 'idx', 'input_ids', 'labels', 'sentence1', 'sentence2', 'token_type_ids'], 'test': ['attention_mask', 'idx', 'input_ids', 'labels', 'sentence1', 'sentence2', 'token_type_ids']}


In [31]:
tokenized_datasets_input = dataset_input.map(tokenize_function,batched=True)
print(tokenized_datasets_input.column_names)

Loading cached processed dataset at C:\Users\liuyw\.cache\huggingface\datasets\json\default-2548fa9cfe6b59ea\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50\cache-15efb4742a9028a6.arrow


{'input': ['attention_mask', 'idx', 'input_ids', 'sentence1', 'sentence2', 'token_type_ids']}


In [32]:
from transformers import DataCollatorWithPadding

In [33]:
data_collator = DataCollatorWithPadding(tokenizer)

In [34]:
from transformers import BertForSequenceClassification

In [35]:
model = BertForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\liuyw/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from c

In [36]:
from transformers import TrainingArguments

In [14]:
training_args = TrainingArguments("test-trainer")

In [15]:
from transformers import Trainer

trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json
Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test-trainer\checkpoint-1000
Configuration saved in test-trainer\checkpoint-1000\config.json
Model weights saved in test-trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.21565924241807727, metrics={'train_runtime': 3013.7602, 'train_samples_per_second': 2.986, 'train_steps_per_second': 0.373, 'total_flos': 591999874560000.0, 'train_loss': 0.21565924241807727, 'epoch': 3.0})

In [18]:
predictions = trainer.predict(tokenized_datasets['test'])
print(predictions.predictions.shape,predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [19]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds = np.argmax(predictions.predictions,axis=1)

metric.compute(predictions=preds,references=predictions.label_ids)

{'f1': 0.9001956947162426}

In [20]:
metric_p = load_metric("precision")

preds_p = np.argmax(predictions.predictions,axis=1)

metric_p.compute(predictions=preds_p,references=predictions.label_ids)

{'precision': 0.8949416342412452}

In [21]:
metric_r = load_metric("recall")

preds_r = np.argmax(predictions.predictions,axis=1)

metric_r.compute(predictions=preds_r,references=predictions.label_ids)

{'recall': 0.905511811023622}

In [22]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.2645178735256195,
 'eval_runtime': 42.245,
 'eval_samples_per_second': 11.836,
 'eval_steps_per_second': 1.491,
 'epoch': 3.0}

In [27]:
predictions_input = trainer.predict(tokenized_datasets_input['input'])
print(predictions.predictions.shape,predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Prediction *****
  Num examples = 38396
  Batch size = 8


(500, 2) (500,)


<h1>Fine Tuning<h1>

In [25]:
training_args_with_epoch = TrainingArguments("test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [27]:
from transformers import Trainer

trainer_eval_epoch = Trainer(
        model,
        training_args_with_epoch,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_eval_epoch.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoin

TrainOutput(global_step=1125, training_loss=0.2048187730577257, metrics={'train_runtime': 2918.5872, 'train_samples_per_second': 3.084, 'train_steps_per_second': 0.385, 'total_flos': 591999874560000.0, 'train_loss': 0.2048187730577257, 'epoch': 3.0})

In [28]:
predictions_eval_epoch = trainer_eval_epoch.predict(tokenized_datasets['test'])
print(predictions_eval_epoch.predictions.shape,predictions_eval_epoch.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [29]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'f1': 0.8949416342412451}

In [31]:
metric_p = load_metric("precision")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric_p.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'precision': 0.8846153846153846}

In [32]:
metric_r = load_metric("recall")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric_r.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'recall': 0.905511811023622}

In [33]:
trainer_eval_epoch.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.4350023865699768,
 'eval_runtime': 40.1733,
 'eval_samples_per_second': 12.446,
 'eval_steps_per_second': 1.568,
 'epoch': 3.0}

<h3>eval steps<h3>

In [36]:
training_args_with_steps = TrainingArguments("test_trainer", evaluation_strategy="steps")

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [37]:
from transformers import Trainer

trainer_eval_steps = Trainer(
        model,
        training_args_with_steps,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_eval_steps.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoin

TrainOutput(global_step=1125, training_loss=0.16231480068630644, metrics={'train_runtime': 2968.0084, 'train_samples_per_second': 3.032, 'train_steps_per_second': 0.379, 'total_flos': 591999874560000.0, 'train_loss': 0.16231480068630644, 'epoch': 3.0})

In [38]:
predictions_eval_steps = trainer_eval_steps.predict(tokenized_datasets['test'])
print(predictions_eval_steps.predictions.shape,predictions_eval_steps.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [39]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_eval_steps = np.argmax(predictions_eval_steps.predictions,axis=1)

metric.compute(predictions=preds_eval_steps,references=predictions_eval_steps.label_ids)

{'f1': 0.8923076923076924}

In [40]:
metric_p = load_metric("precision")

preds_eval_steps = np.argmax(predictions_eval_steps.predictions,axis=1)

metric_p.compute(predictions=preds_eval_steps,references=predictions_eval_steps.label_ids)

{'precision': 0.8721804511278195}

In [41]:
metric_r = load_metric("recall")

preds_eval_steps = np.argmax(predictions_eval_steps.predictions,axis=1)

metric_r.compute(predictions=preds_eval_steps,references=predictions_eval_steps.label_ids)

{'recall': 0.9133858267716536}

In [42]:
trainer_eval_steps.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.5802590847015381,
 'eval_runtime': 46.0255,
 'eval_samples_per_second': 10.864,
 'eval_steps_per_second': 1.369,
 'epoch': 3.0}

<h3>eval epoch<h3>

In [15]:
training_args_with_epoch = TrainingArguments("test_trainer", evaluation_strategy="epoch")

In [16]:
from transformers import Trainer

trainer_eval_epoch = Trainer(
        model,
        training_args_with_epoch,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_eval_epoch.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoin

TrainOutput(global_step=1125, training_loss=0.23919410875108507, metrics={'train_runtime': 2669.8239, 'train_samples_per_second': 3.371, 'train_steps_per_second': 0.421, 'total_flos': 591999874560000.0, 'train_loss': 0.23919410875108507, 'epoch': 3.0})

In [45]:
predictions_eval_epoch = trainer_eval_epoch.predict(tokenized_datasets['test'])
print(predictions_eval_epoch.predictions.shape,predictions_eval_epoch.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [46]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'f1': 0.9090909090909092}

In [47]:
metric_p = load_metric("precision")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric_p.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'precision': 0.8759124087591241}

In [48]:
metric_r = load_metric("recall")

preds_eval_epoch = np.argmax(predictions_eval_epoch.predictions,axis=1)

metric_r.compute(predictions=preds_eval_epoch,references=predictions_eval_epoch.label_ids)

{'recall': 0.9448818897637795}

In [49]:
trainer_eval_epoch.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.596695065498352,
 'eval_runtime': 39.465,
 'eval_samples_per_second': 12.669,
 'eval_steps_per_second': 1.596,
 'epoch': 3.0}

<h3>Logging epoch<h3>

In [50]:
training_args_logging_with_epoch = TrainingArguments("test_trainer", logging_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
from transformers import Trainer

trainer_logging_epoch = Trainer(
        model,
        training_args_logging_with_epoch,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_logging_epoch.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.1310844285753038, metrics={'train_runtime': 2795.1963, 'train_samples_per_second': 3.22, 'train_steps_per_second': 0.402, 'total_flos': 591999874560000.0, 'train_loss': 0.1310844285753038, 'epoch': 3.0})

In [52]:
predictions_logging_epoch = trainer_logging_epoch.predict(tokenized_datasets['test'])
print(predictions_logging_epoch.predictions.shape,predictions_logging_epoch.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [53]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_logging_epoch = np.argmax(predictions_logging_epoch.predictions,axis=1)

metric.compute(predictions=preds_logging_epoch,references=predictions_logging_epoch.label_ids)

{'f1': 0.9027237354085603}

In [54]:
metric_p = load_metric("precision")

preds_logging_epoch = np.argmax(predictions_logging_epoch.predictions,axis=1)

metric_p.compute(predictions=preds_logging_epoch,references=predictions_logging_epoch.label_ids)

{'precision': 0.8923076923076924}

In [55]:
metric_r = load_metric("recall")

preds_logging_epoch = np.argmax(predictions_logging_epoch.predictions,axis=1)

metric_r.compute(predictions=preds_logging_epoch,references=predictions_logging_epoch.label_ids)

{'recall': 0.9133858267716536}

In [56]:
trainer_logging_epoch.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.57377028465271,
 'eval_runtime': 38.5068,
 'eval_samples_per_second': 12.985,
 'eval_steps_per_second': 1.636,
 'epoch': 3.0}

<h3>Logging steps<h3>

In [57]:
training_args_logging_with_steps = TrainingArguments("test_trainer", logging_strategy="steps")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
from transformers import Trainer

trainer_logging_steps = Trainer(
        model,
        training_args_logging_with_steps,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_logging_steps.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.09461118613349066, metrics={'train_runtime': 2687.1116, 'train_samples_per_second': 3.349, 'train_steps_per_second': 0.419, 'total_flos': 591999874560000.0, 'train_loss': 0.09461118613349066, 'epoch': 3.0})

In [60]:
predictions_logging_steps = trainer_logging_steps.predict(tokenized_datasets['test'])
print(predictions_logging_steps.predictions.shape,predictions_logging_steps.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [61]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_logging_steps = np.argmax(predictions_logging_steps.predictions,axis=1)

metric.compute(predictions=preds_logging_steps,references=predictions_logging_steps.label_ids)

{'f1': 0.8944337811900193}

In [62]:
metric_p = load_metric("precision")

preds_logging_steps = np.argmax(predictions_logging_steps.predictions,axis=1)

metric_p.compute(predictions=preds_logging_steps,references=predictions_logging_steps.label_ids)

{'precision': 0.8726591760299626}

In [63]:
metric_r = load_metric("recall")

preds_logging_steps = np.argmax(predictions_logging_steps.predictions,axis=1)

metric_r.compute(predictions=preds_logging_steps,references=predictions_logging_steps.label_ids)

{'recall': 0.9173228346456693}

In [64]:
trainer_logging_steps.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.6918169260025024,
 'eval_runtime': 41.4748,
 'eval_samples_per_second': 12.056,
 'eval_steps_per_second': 1.519,
 'epoch': 3.0}

<h3>save epoch<h3>

In [37]:
training_args_save_with_epoch = TrainingArguments("test_trainer", save_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
from transformers import Trainer

trainer_save_epoch = Trainer(
        model,
        training_args_save_with_epoch,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_save_epoch.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test_trainer\checkpoint-375
Configuration saved in test_trainer\checkpoint-375\config.json
Model weights saved in test_trainer\checkpoint-375\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-375\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-375\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-750
Configuration saved in test_trainer\checkpoint-750\config.json
Model weights saved in test_trainer\checkpoint-750\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-750\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-750\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-1125
Configuration saved in test_trainer\checkpoint-1125\config.json
Model weights saved in test_trainer\checkpoint-1125\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-1125\tokenizer_config.json
Special tokens file saved in t

TrainOutput(global_step=1125, training_loss=0.2841971469455295, metrics={'train_runtime': 2967.7883, 'train_samples_per_second': 3.033, 'train_steps_per_second': 0.379, 'total_flos': 591999874560000.0, 'train_loss': 0.2841971469455295, 'epoch': 3.0})

In [14]:
predictions_save_epoch = trainer_save_epoch.predict(tokenized_datasets['test'])
print(predictions_save_epoch.predictions.shape,predictions_save_epoch.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [15]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_save_epoch = np.argmax(predictions_save_epoch.predictions,axis=1)

metric.compute(predictions=preds_save_epoch,references=predictions_save_epoch.label_ids)

{'f1': 0.9023437500000001}

In [16]:
metric_p = load_metric("precision")

preds_save_epoch = np.argmax(predictions_save_epoch.predictions,axis=1)

metric_p.compute(predictions=preds_save_epoch,references=predictions_save_epoch.label_ids)

{'precision': 0.8953488372093024}

In [17]:
metric_r = load_metric("recall")

preds_save_epoch = np.argmax(predictions_save_epoch.predictions,axis=1)

metric_r.compute(predictions=preds_save_epoch,references=predictions_save_epoch.label_ids)

{'recall': 0.9094488188976378}

In [18]:
trainer_save_epoch.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.29276323318481445,
 'eval_runtime': 39.1941,
 'eval_samples_per_second': 12.757,
 'eval_steps_per_second': 1.607,
 'epoch': 3.0}

<h3>save steps<h3>

In [19]:
training_args_save_with_steps = TrainingArguments("test_trainer", save_strategy="steps")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
from transformers import Trainer

trainer_save_steps = Trainer(
        model,
        training_args_save_with_steps,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        )
trainer_save_steps.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json
Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.1560798297458225, metrics={'train_runtime': 2701.3104, 'train_samples_per_second': 3.332, 'train_steps_per_second': 0.416, 'total_flos': 591999874560000.0, 'train_loss': 0.1560798297458225, 'epoch': 3.0})

In [21]:
predictions_save_steps = trainer_save_steps.predict(tokenized_datasets['test'])
print(predictions_save_steps.predictions.shape,predictions_save_steps.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


(500, 2) (500,)


In [22]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

preds_save_steps = np.argmax(predictions_save_steps.predictions,axis=1)

metric.compute(predictions=preds_save_steps,references=predictions_save_steps.label_ids)

{'f1': 0.8978805394990366}

In [23]:
metric_p = load_metric("precision")

preds_save_steps = np.argmax(predictions_save_steps.predictions,axis=1)

metric_p.compute(predictions=preds_save_steps,references=predictions_save_steps.label_ids)

{'precision': 0.879245283018868}

In [24]:
metric_r = load_metric("recall")

preds_save_steps = np.argmax(predictions_save_steps.predictions,axis=1)

metric_r.compute(predictions=preds_save_steps,references=predictions_save_steps.label_ids)

{'recall': 0.9173228346456693}

In [25]:
trainer_save_steps.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.6130175590515137,
 'eval_runtime': 39.4215,
 'eval_samples_per_second': 12.683,
 'eval_steps_per_second': 1.598,
 'epoch': 3.0}

<h1>Predictions<h1>

In [ ]:
predictions_input = trainer_save_epoch.predict(tokenized_datasets_input['input'])
#print(predictions.predictions.shape,predictions.label_ids.shape)

In [ ]:
import numpy as np
preds = np.argmax(predictions_input.predictions,axis=1)
preds

In [22]:
with open('../Datasets/final_Bert_gleif_bisnode_FULL.txt', 'w') as f:
    for item in preds:
        f.write("%s\n" % item)